In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pyomo.environ as pyo
from pyomo.contrib.parmest import parmest  # import parmest
from pyomo.contrib.parmest.experiment import Experiment
from pyomo.contrib.doe import DesignOfExperiments
import pandas as pd
from itertools import product

# Experiment Class

In [3]:
class ReactionVelocity(Experiment):

    # Defining the constructor for our model
    def __init__(self, data, theta_initial=None, scale_params=True):
        """
        Arguments:
            data: data from our experiment. type: 'dict'
            theta_initial: initial guess of the parameter values, dtype: dict. pass the values as theta_initial = {1 : <theta_1 initial value>, 2 : <theta_2 initial value>}
                default:  {1: 100, 2: 0.05}

        """
        self.conc = data["substrate_concentration"]
        self.vel = data["avg_velocity"]
        self.model = None
        self.theta_initial = theta_initial
        if self.theta_initial is None:
            self.theta_initial = {
                1: 100,
                2: 0.02,
            }  # default initial guess of theta[1] & theta[2]
        else:
            self.theta_initial = theta_initial

        if scale_params:
            self.theta_1_scale = 200
            self.theta_2_scale = 0.050
        else:
            self.theta_1_scale = 1
            self.theta_2_scale = 1

    # Creating the get_labeled_model which is a must for ``DOE`` and ``ParmEst``
    def get_labeled_model(self):
        if self.model is None:
            self.create_model()
            self.label_model()
            self.finalize_model()
        return self.model

    def create_model(self):
        """
        Here, we will create different variables, parameters, and constraints
        """
        m = self.model = pyo.ConcreteModel()

        # theta_1
        m.theta_1 = pyo.Var(
            domain=pyo.NonNegativeReals, initialize=self.theta_initial[1]
        )
        m.theta_2 = pyo.Var(
            domain=pyo.NonNegativeReals, initialize=self.theta_initial[2]
        )

        # substrate concentration (x) as a param, since the data is given
        m.x = pyo.Var(domain=pyo.NonNegativeReals, initialize=self.conc)
        m.reaction_velocity = pyo.Var(domain=pyo.NonNegativeReals)

        @m.Constraint()  # since the ``Constraint`` is not a set of anything, so there is no argument
        def vel_con(m):
            return m.reaction_velocity == m.theta_1 * self.theta_1_scale * m.x / (
                self.theta_2_scale * m.theta_2 + m.x
            )

        # ======================================
        # Objective function
        """
        For some reason the``obj_function = "SSE"``in parmest ``Estimator`` does not work for LR ratio.
        That's why I have created the ``Total_Cost_Obj`` to minimize the SSE. You can ignore this and use default "SSE". 
        Both ``FirstStageCost`` and ``SecondStageCost`` are required for the ``Objective``, otherwise it will show ``AttributeError``
        """

        # # Stage-specific cost computation
        # m.FirstStageCost = pyo.Expression(initialize = 0)
        # m.SecondStageCost = pyo.Expression(expr = (self.vel - m.reaction_velocity) ** 2)

        # m.Total_Cost_Obj = pyo.Objective(expr = (m.FirstStageCost + m.SecondStageCost), sense = pyo.minimize)
        # # ======================================

        # m.pprint()

        return m

    def finalize_model(self):
        """
        Finalizing the model. Here, we will set the experimental conditions (e.g, initial conditions),
        fixing the parameter values (if needed), update `t` values, and discretize the model (if model is dynamic).
        It makes a solvable model.
        """
        m = self.model

        # fixing the parameters
        m.theta_1.fix(self.theta_initial[1])
        m.theta_2.fix(self.theta_initial[2])

        # Add lower and upper bound to substrate concentration (control variable)
        m.x.setlb(0)
        m.x.setub(2)

        # fixing the substrate concentration (control variable)
        m.x.fix(self.conc)

        return m

    def label_model(self):
        """
        The model is updated with outputs, and unknown parameters. This makes the model labeled with full experiment.
        In `ParmEst` output (given data) is the most important. For `DOE` input is most important.
        """
        m = self.model

        m.experiment_outputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.experiment_outputs.update(
            [(m.reaction_velocity, self.vel)]
        )  # Pass the data as a list of `tuple`
        # If we only use ``DOE``, we could use ``m.experiment_ouputs.update([(m.x, None)])``.
        # Output is not important for ``DOE``

        m.experiment_inputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        # m.experiment_inputs[m.x] = self.conc
        m.experiment_inputs.update([(m.x, self.conc)])
        # If we only use ``DOE``, we could use ``m.experiment_inputs.update([(m.x, None)])``

        m.unknown_parameters = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.unknown_parameters.update((p, pyo.value(p)) for p in [m.theta_1, m.theta_2])
        # m.unknown_parameters[m.theta_1]= self.theta_initial[1]
        # m.unknown_parameters[m.theta_2]= self.theta_initial[2]

        m.measurement_error = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        # m.measurement_error[m.reaction_velocity] = 5
        m.measurement_error.update([(m.reaction_velocity, 5)])

        return m

In [4]:
theta_initial = {1: 1.036914, 2: 1.170216}

# True parameters

In [5]:
# Generate Data
theta_true = [212.71, 0.0645]  # True parameter values
def michaelis_menten_model_data(x, theta=theta_true):
    np.random.seed(42)
    """
    Michaelis-Menten model function to generate data.
    Vmax, Km = theta
    x: substrate concentration. ppm
    -----------------------------------------
    Returns the reaction rate. counts/min^2
    """
    Vmax, Km = theta
    f = (Vmax * x) / (Km + x) + np.random.normal(0, 5)  # Adding some noise
    return f

In [6]:
subs_conc = np.array([0.02, 0.06])
rxn_rate = michaelis_menten_model_data(subs_conc)
rxn_rate

# Organizing the data in a ``list`` consisting of ``dict``s of ``one key : one value``
data_treated = [
    {"substrate_concentration": sc, "avg_velocity": rr}
    for sc, rr in zip(subs_conc, rxn_rate)
]

In [7]:
FIM = np.zeros((2, 2))
for i in range(len(data_treated)):
    # Create a ReactorVelocity object for each experiment
    experiment = ReactionVelocity(
        data_treated[i],
        theta_initial=theta_initial,
    )

    scale_nominal_param_value = False
    doe_obj = DesignOfExperiments(
        experiment, scale_nominal_param_value=scale_nominal_param_value, tee=False
    )
    # I haven't passed the other arguments, because the defaults values are used in the doe example.

    FIM+= doe_obj.compute_FIM()


/var/folders/__/5p42xpfs2tj5j1_sgllkjwxh0000gs/T/ipykernel_26695/2746755250.py:10: DeprecationWarning: The 'experiment' parameter in DesignOfExperiments is deprecated and will be removed in a future version. Please use 'experiment_list' instead.
  doe_obj = DesignOfExperiments(
/var/folders/__/5p42xpfs2tj5j1_sgllkjwxh0000gs/T/ipykernel_26695/2746755250.py:10: DeprecationWarning: The 'experiment' parameter in DesignOfExperiments is deprecated and will be removed in a future version. Please use 'experiment_list' instead.
  doe_obj = DesignOfExperiments(


In [8]:
FIM

array([[ 513.94551291, -247.98151208],
       [-247.98151208,  123.76849951]])

# Optimize single experiment

In [9]:
experiment = ReactionVelocity(
    data_treated[0],
    theta_initial=theta_initial,
)
mm_doe = DesignOfExperiments(experiment_list=experiment, prior_FIM=FIM, tee=True)

mm_doe.run_doe()

Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for

In [10]:
# Show the results
mm_doe.results

{'Solver Status': <SolverStatus.ok: 'ok'>,
 'Termination Condition': <TerminationCondition.optimal: 'optimal'>,
 'Termination Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found',
 'FIM': [[np.float64(2024.2818489449105), np.float64(-286.02088258064236)],
  [np.float64(-286.02088258064236), np.float64(124.72656008708256)]],
 'Sensitivity Matrix': [[np.float64(194.31522946206601),
   np.float64(-4.894028456299619)]],
 'Experiment Design': [1.9999997277237545],
 'Experiment Design Names': ['x'],
 'Experiment Outputs': [201.68967002428673],
 'Experiment Output Names': ['reaction_velocity'],
 'Unknown Parameters': [1.0379509139999998, 1.170216],
 'Unknown Parameter Names': ['theta_1', 'theta_2'],
 'Measurement Error': [201.68967002428673],
 'Measurement Error Names': ['reaction_velocity'],
 'Prior FIM': [[np.float64(513.9455129090959),
   np.float64(-247.98151208145276)],
  [np.float64(-247.9815120814527), np.float64(123.76849950583974)]],
 'Objective expression': 'determinant',
 'log10 A-

# Optimize 2 experiments simultaneously

In [12]:
experiment = ReactionVelocity(
    data_treated[0],
    theta_initial=theta_initial,
)
# Let's be lazy and use the same experiment twice to see what happens
mm_doe = DesignOfExperiments(experiment_list=[experiment, experiment], prior_FIM=None, tee=True)

mm_doe.optimize_experiments()

Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for

In [13]:
mm_doe.results

{'Solver Status': <SolverStatus.ok: 'ok'>,
 'Termination Condition': <TerminationCondition.optimal: 'optimal'>,
 'Termination Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found',
 'Scenarios': [{'Total FIM': [[1887.920227388308, 0.0],
    [-210.08064290027085, 79.34642683360543]],
   'log10 A-opt': np.float64(-1.7300714107793733),
   'log10 pseudo A-opt': np.float64(3.293863230589532),
   'log10 D-opt': np.float64(5.023934641368904),
   'log10 E-opt': np.float64(1.742446264680762),
   'FIM Condition Number': np.float64(34.59729239764324),
   'Experiments': [{'Experiment Design': [0.05527650859117343],
     'Experiment Design Names': ['x'],
     'Experiment Outputs': [100.8448188559896],
     'Experiment Output Names': ['reaction_velocity'],
     'Unknown Parameters': [1.0379509139999998, 1.170216],
     'Unknown Parameter Names': ['theta_1', 'theta_2'],
     'Measurement Error': [100.8448188559896],
     'Measurement Error Names': ['reaction_velocity'],
     'FIM': [[377.5839630250313